In [1]:
from modelUnetResnet50 import *
from data import *
import os, os.path

vid="v009_5" #version id should match the file number. Last number shows the cross-validation fold number
# Ran on DellWS with GeForce RTX3060 GPU

# Count the number of train and valid files
train_dir = 'mass_seg_08/train0'+vid[-1]+'/mg'
train_count=len([name for name in os.listdir(train_dir) if os.path.isfile(os.path.join(train_dir, name))])

valid_dir = 'mass_seg_08/valid0'+vid[-1]+'/mg'
valid_count=len([name for name in os.listdir(valid_dir) if os.path.isfile(os.path.join(valid_dir, name))])

### Train with data generator

In [2]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger

#Data augmentation
data_gen_args = dict(rotation_range=90,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='wrap')

# SETTINGS ***
batch_size=2
learning_rate=1e-4

train_gen = trainGenerator(batch_size,'mass_seg_08/train0'+vid[-1],'mg','mask',data_gen_args,save_to_dir = None)
valid_gen = trainGenerator(batch_size,'mass_seg_08/valid0'+vid[-1],'mg','mask',data_gen_args,save_to_dir = None)

# train_count images are used for training, valid_count images for validating
train_steps = train_count//batch_size
valid_steps = valid_count//batch_size

# SETTINGS ***
loss=dice_loss
steps_per_epoch=2.5*train_steps
num_epochs=100

model = build_resnet50_unet()

opt = tf.keras.optimizers.Adam(learning_rate)
metrics = ["acc", dice_coef, iou]
model.compile(loss=loss, optimizer=opt, metrics=metrics)

callbacks = [
             ModelCheckpoint('files_mass_seg_xval/unet_mass_seg_'+vid+'.hdf5', verbose=1, save_best_model=True),
             ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.1, verbose=1, min_lr=1e-8),
             CSVLogger("files_mass_seg_xval/data_"+vid+".csv"),
             EarlyStopping(monitor="val_loss", patience=5, verbose=1)
            ]

model.fit(train_gen, validation_data=valid_gen, steps_per_epoch=steps_per_epoch, validation_steps=valid_steps, 
                    epochs=num_epochs, callbacks=callbacks)


Found 283 images belonging to 1 classes.
Found 283 images belonging to 1 classes.
Epoch 1/100
353/352 [==============================] - ETA: 0s - loss: 0.8514 - acc: 0.9141 - dice_coef: 0.1485 - iou: 0.0855Found 69 images belonging to 1 classes.
Found 69 images belonging to 1 classes.

Epoch 1: saving model to files_mass_seg_xval\unet_mass_seg_v009_5.hdf5
352/352 [==============================] - 349s 770ms/step - loss: 0.8514 - acc: 0.9141 - dice_coef: 0.1485 - iou: 0.0855 - val_loss: 0.9632 - val_acc: 0.9817 - val_dice_coef: 0.0368 - val_iou: 0.0189 - lr: 1.0000e-04
Epoch 2/100
353/352 [==============================] - ETA: 0s - loss: 0.7594 - acc: 0.9680 - dice_coef: 0.2402 - iou: 0.1449
Epoch 2: saving model to files_mass_seg_xval\unet_mass_seg_v009_5.hdf5
352/352 [==============================] - 242s 685ms/step - loss: 0.7594 - acc: 0.9680 - dice_coef: 0.2402 - iou: 0.1449 - val_loss: 0.9645 - val_acc: 0.9814 - val_dice_coef: 0.0355 - val_iou: 0.0184 - lr: 1.0000e-04
Epoch 3/

Epoch 19: early stopping


### validate your model and save predicted results

In [4]:
validGene = testGenerator("mass_seg_08/valid0"+vid[-1]+"/pred",num_image=valid_count)
model.load_weights("files_mass_seg_xval/unet_mass_seg_"+vid+".hdf5")
results = model.predict(validGene,valid_count,verbose=1)
saveResult("mass_seg_08/valid0"+vid[-1]+"/pred",results,vid[:4])

69/69 [==============================] - 16s 239ms/step


D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_08/valid05/pred\1_predict_v009.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_08/valid05/pred\3_predict_v009.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_08/valid05/pred\4_predict_v009.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_08/valid05/pred\5_predict_v009.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_08/valid05/pred\9_predict_v009.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"